In [1]:
from Bug import Bug
from mongoengine import connect
import math
import json
import copy

connect('bug_reports_db', host='127.0.0.1', port=27017)

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [2]:
relations = ['assigned_to-priority', 'assigned_to-status', 'component-product', 'priority-severity', 'platform-op_sys']

bugs_relations = []
temp_bug_init = {}

for relation in relations:
  temp_bug_init[relation] = 0

def calc_relations(history):
  current_relations = copy.deepcopy(temp_bug_init)

  for changes in history:
    current_fields_set = set()

    for change in changes['changes']:
      field_name = change['field_name']

      current_fields_set.add(field_name)
    
    verify_relations(current_relations, current_fields_set)

  return current_relations

def verify_relations(current_relations, current_fields_set):
  for relation in relations:
    fields_to_verify_set = set(relation.split('-'))
    
    if fields_to_verify_set.issubset(current_fields_set):
      current_relations[relation] += 1


In [3]:
BUGS_LIMIT = 50000
total_bugs = Bug.objects.count()
bugs_per_iteration = math.ceil(total_bugs/BUGS_LIMIT)

print(f'Iterarions needed: {bugs_per_iteration}')

for i in range(bugs_per_iteration):
  skip_quantity = i * BUGS_LIMIT
  result = Bug.objects.only('bug_id', 'history').limit(BUGS_LIMIT).skip(skip_quantity)

  bugs = [bug.to_mongo().to_dict() for bug in result]

  print(f'Qnt. de bugs nesta iteração: {len(bugs)}')

  for bug in bugs:
    temp_bug = calc_relations(bug['history'])
    temp_bug['bug_id'] = bug['bug_id']
    bugs_relations.append(temp_bug)
    
  print(f"Iteration {i}")
                    

Iterarions needed: 14
Qnt. de bugs nesta iteração: 50000
Iteration 0
Qnt. de bugs nesta iteração: 50000
Iteration 1
Qnt. de bugs nesta iteração: 50000
Iteration 2
Qnt. de bugs nesta iteração: 50000
Iteration 3
Qnt. de bugs nesta iteração: 50000
Iteration 4
Qnt. de bugs nesta iteração: 50000
Iteration 5
Qnt. de bugs nesta iteração: 50000
Iteration 6
Qnt. de bugs nesta iteração: 50000
Iteration 7
Qnt. de bugs nesta iteração: 50000
Iteration 8
Qnt. de bugs nesta iteração: 50000
Iteration 9
Qnt. de bugs nesta iteração: 50000
Iteration 10
Qnt. de bugs nesta iteração: 50000
Iteration 11
Qnt. de bugs nesta iteração: 50000
Iteration 12
Qnt. de bugs nesta iteração: 40817
Iteration 13


In [4]:
with open('processed_bugs_counting_relations_2.json', 'w') as output:
  json.dump(bugs_relations, output)